if you want to know more about  preprocess function go to this link 
https://github.com/Faizan-E-Mustafa/Udacity-Intro-to-ML-Exercises/blob/master/Naive%20bayes%20(Additional%20learning%20resources%20included).ipynb

In [1]:

import pickle

import numpy as np
from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif
def preprocess(per, words_file = "tools/word_data.pkl", authors_file="tools/email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "rb")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "rb")
    word_data = pickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size= 0.14, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    #print(vectorizer.vocabulary_)
    #print(vectorizer.idf_)
    
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=per)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print ("no. of Chris training emails:", sum(labels_train))
    print ("no. of Sara training emails:", len(labels_train)-sum(labels_train))
    
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test


/home/faizan/.virtualenvs/keras_tf/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


* Q1 :find the number of features in your data.  The data is organized into a numpy array where the number of rows is the number of data points and the number of columns is the number of features; so to extract this number, use a line of code like len(features_train[0])

find the line of code that looks like this:     selector = SelectPercentile(f_classif, percentile=1)  Change percentile from 10 to 1.

* Q2: What’s the number of features now?


* Q3: What do you think SelectPercentile is doing?  Would a large value for percentile lead to a more complex or less complex decis , all other things being equal?
  
  
* Q4: What’s the accuracy when percentile = 1?


In [4]:
# decision tree

### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
print("Decion tree with percentile=10")
features_train, features_test, labels_train, labels_test = preprocess(per = 10)
from time import time
from sklearn import tree



clf = tree.DecisionTreeClassifier(min_samples_split=40)

t0 = time()
clf.fit(features_train,labels_train)
print ("training time:", round(time()-t0, 3), "s")

t0 = time()
pred = clf.predict(features_test)
print ("test time:", round(time()-t0, 3), "s")

from sklearn.metrics import accuracy_score
acc = accuracy_score(pred, labels_test)
print("accuracy percentile=10:",acc)


print("Q1:", len(features_train[0]))

print("Decion tree with percentile=1")

features_train, features_test, labels_train, labels_test = preprocess(per = 1)
clf = tree.DecisionTreeClassifier(min_samples_split=40)

t0 = time()
clf.fit(features_train,labels_train)
print ("training time:", round(time()-t0, 3), "s")

t0 = time()
pred = clf.predict(features_test)
print ("test time:", round(time()-t0, 3), "s")


acc = accuracy_score(pred, labels_test)


print("Q2:", len(features_train[0]))
print("Q3:", "large values percentile will select large numbr of features and large numbr of features result in a more complex decion tree which is more prone to overfitting")
from sklearn.metrics import accuracy_score
acc = accuracy_score(pred, labels_test)
print("Q4:accuracy percentile=1:", acc)

Decion tree with percentile=10
no. of Chris training emails: 7571
no. of Sara training emails: 7546
training time: 115.316 s
test time: 0.074 s
accuracy percentile=10: 0.982933766761479
Q1: 3749
Decion tree with percentile=1
no. of Chris training emails: 7571
no. of Sara training emails: 7546
training time: 8.092 s
test time: 0.004 s
Q2: 375
Q3: large values percentile will select large numbr of features and large numbr of features result in a more complex decion tree which is more prone to overfitting
Q4:accuracy percentile=1: 0.9650548557496953
